In [6]:
import pandas as pd
import requests
import numpy as np
import time

# Get cities coordinates

We will retrieve the coordinates of the top 35 cities using the Nominatim API

In [27]:
cities = pd.read_csv('input/top35-cities.csv', header=None, names=['city', 'lat', 'lon'])

In [ ]:
# Get location and data
for index, row in cities.iterrows():
    try :
        response = requests.get(f'https://nominatim.openstreetmap.org/search?q={row["city"]}&country=france&format=json')
        first_result = response.json()[0]
        lat = first_result['lat']
        lon = first_result['lon']

    except:
        lat = None
        lon = None
        print(f'Could not find coordinates for {row["city"]}')
    
    cities.loc[index, 'lat'] = lat
    cities.loc[index, 'lon'] = lon

In [29]:
cities.head()

,city,lat,lon
0,Mont Saint Michel,48.6359541,-1.511459954959514
1,St Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


# Collect weather data
We will retrieve the weather data of the top 35 cities for the next 7 days using the Openweather API.

In [30]:
# Openweathers API key
openweather_apikey = 'YOUR_KEY'

In [31]:
# Create the weather columns
for column in ['mean_feels_like', 'mean_pop']:
    cities[column]= None
    #cities[column] = cities[column].astype('object')

In [54]:
for index, row in cities.iterrows():
    try:
        # API call
        response = requests.get(f'https://api.openweathermap.org/data/2.5/onecall?lat={row["lat"]}&lon={row["lon"]}&exclude=current,hourly,minutely&units=metric&appid={openweather_apikey}')
        weather = response.json()

        # Extract interesting data 
        ## Round the temperature to the closest integer
        mean_feels_like = round(np.mean([day['feels_like']['day'] for day in weather['daily']]))
        
        ## Round the probability of precipitations to the nearest 0.5
        mean_pop = round(np.mean([day['pop'] for day in weather['daily']]) * 2 ) / 2
        
    
    except:
        mean_feels_like = []
        mean_pop = []
        print(f'Could not find weather data for {row["city"]}')
    
    # Insert into the dataframe
    cities.at[index,'mean_feels_like'] = mean_feels_like
    cities.at[index,'mean_pop'] = mean_pop


In [62]:
cities.sort_values(by=['mean_pop', 'mean_feels_like'], ascending=[True, False], inplace=True, ignore_index=True)

In [64]:
# Export data for later use
cities.to_csv('results/cities.csv', header=True, index=True, index_label='city_id')